In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
import random
import torch as t

In [2]:
lemmatizer = WordNetLemmatizer()
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))


pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

In [4]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
random.shuffle(training)

In [5]:
train_x,train_y = [],[]

for a in training:
    train_x.append(a[0])
    train_y.append(a[1])

train_x = t.tensor(train_x)
train_y = t.tensor(train_y)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# class ChatbotRNN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(ChatbotRNN, self).__init__()

#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.output_size = output_size

#         self.embedding = nn.Embedding(input_size, hidden_size)
#         self.rnn = nn.RNN(hidden_size, hidden_size)
#         self.decoder = nn.Linear(hidden_size, output_size)

#     def forward(self, input, hidden):
#         embeds = self.embedding(input)
#         outputs, hidden = self.rnn(embeds, hidden)
#         outputs = self.decoder(outputs)
#         return outputs, hidden

# def train(model, data , epochs , loss_fn):
#     optimizer = optim.Adam(model.parameters())
#     for epoch in range(epochs):
#         for input, output in data:
#             hidden = torch.zeros(1, model.hidden_size)
#             output_pred, hidden = model(input, hidden)
#             loss = loss_fn(output_pred, output)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

# def evaluate(model, data):
#     correct = 0
#     total = 0
#     for input, output in data:
#         hidden = torch.zeros(1, model.hidden_size)
#         output_pred, hidden = model(input, hidden)
#         pred = output_pred.argmax(1)
#         correct += (pred == output).sum()
#         total += 1
#     return correct / total

In [23]:
class NeuralNet(t.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = t.nn.Linear(input_size, hidden_size)
        self.l2 = t.nn.RNN(hidden_size, hidden_size)
        self.l3 = t.nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out,hid = self.l2(out)
        out = self.l3(out)
        out = t.nn.functional.softmax(out)
        return out

In [24]:
model = NeuralNet(len(train_x[0]), 16, len(train_y[0]))
loss_fn = t.nn.CrossEntropyLoss()
optimizer = t.optim.Adam(model.parameters(), lr=0.1)

In [25]:
for _ in range(100):
    for a,b in zip(train_x,train_y):
        y_pred = model(a.view(1, -1).float())
        loss = loss_fn(y_pred, b.view(1,-1).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if _ % 100 == 99:
            print(t, loss.item())

C:\Users\NK\AppData\Local\Temp\ipykernel_1096\4152925018.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = t.nn.functional.softmax(out)


<module 'torch' from 'c:\\Users\\NK\\.conda\\envs\\project\\lib\\site-packages\\torch\\__init__.py'> 0.7436683177947998
<module 'torch' from 'c:\\Users\\NK\\.conda\\envs\\project\\lib\\site-packages\\torch\\__init__.py'> 1.743666172027588
<module 'torch' from 'c:\\Users\\NK\\.conda\\envs\\project\\lib\\site-packages\\torch\\__init__.py'> 0.7436683177947998
<module 'torch' from 'c:\\Users\\NK\\.conda\\envs\\project\\lib\\site-packages\\torch\\__init__.py'> 0.7436683177947998
<module 'torch' from 'c:\\Users\\NK\\.conda\\envs\\project\\lib\\site-packages\\torch\\__init__.py'> 0.7436684370040894
<module 'torch' from 'c:\\Users\\NK\\.conda\\envs\\project\\lib\\site-packages\\torch\\__init__.py'> 0.7436684370040894
<module 'torch' from 'c:\\Users\\NK\\.conda\\envs\\project\\lib\\site-packages\\torch\\__init__.py'> 0.7442768812179565
<module 'torch' from 'c:\\Users\\NK\\.conda\\envs\\project\\lib\\site-packages\\torch\\__init__.py'> 1.7436604499816895
<module 'torch' from 'c:\\Users\\NK\\.con

In [26]:
t.save(model.state_dict(), 'model/rnn_model.pth')